In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import geopandas as gpd
from datetime import datetime

# ==============================================================================
# 1. CẤU HÌNH KẾT NỐI (IP Tailscale của bạn)
# ==============================================================================
DB_HOST = "100.117.51.34"

# Nguồn Ops (Postgres)
URL_OPS = f"postgresql://postgres:hung12345@{DB_HOST}:5433/Uber_ops"
# Nguồn CRM (MariaDB)
URL_CRM = f"mysql+pymysql://root:hung12345@{DB_HOST}:3307/Uber_crm"
# Đích DWH (Postgres)
URL_DWH = f"postgresql://postgres:hung12345@{DB_HOST}:5432/Uber_data_warehouse"

engine_ops = create_engine(URL_OPS)
engine_crm = create_engine(URL_CRM)
engine_dwh = create_engine(URL_DWH)

print("🔗 Đã kết nối 3 Database thành công!")

# ==============================================================================
# 2. HÀM TẠO DIM_DATE (Tự động sinh từ 2023 đến 2030)
# ==============================================================================
def populate_dim_date():
    print("\n📅 Đang tạo dữ liệu DimDate...")
    
    start_date = "2023-01-01"
    end_date = "2030-12-31"
    
    # Tạo dãy ngày
    date_range = pd.date_range(start=start_date, end=end_date)
    df = pd.DataFrame(date_range, columns=['fulldate'])
    
    # Tính toán các cột
    df['datekey'] = df['fulldate'].dt.strftime('%Y%m%d').astype(int)
    df['calendaryear'] = df['fulldate'].dt.year
    df['calendarquarter'] = df['fulldate'].dt.to_period('Q').astype(str)
    df['monthnumberofyear'] = df['fulldate'].dt.month
    df['monthname'] = df['fulldate'].dt.month_name()
    df['calendaryearmonth'] = df['fulldate'].dt.strftime('%Y-%m')
    df['daynumberofmonth'] = df['fulldate'].dt.day
    df['daynumberofyear'] = df['fulldate'].dt.dayofyear
    df['daynameofweek'] = df['fulldate'].dt.day_name()
    df['weeknumberofyear'] = df['fulldate'].dt.isocalendar().week
    df['isweekday'] = df['fulldate'].dt.dayofweek < 5 # 0-4 là T2-T6
    df['isholiday'] = False # Mặc định False, update sau nếu cần
    df['holidayname'] = None

    # Đẩy vào DWH
    try:
        df.columns = df.columns.str.lower() # Map với tên cột trong DB
        df.to_sql('dimdate', engine_dwh, if_exists='append', index=False, chunksize=5000)
        print(f"✅ Đã nạp {len(df)} dòng vào DimDate.")
    except Exception as e:
        print(f"⚠️ DimDate có thể đã có dữ liệu: {e}")

def populate_dim_location():
    print("\n🌍 Đang nạp DimLocation từ Ops...")
    # 1. Đọc từ Ops
    sql = "SELECT zone_id, zone_name, borough, zone_geom FROM taxi_zones"
    gdf = gpd.read_postgis(sql, engine_ops, geom_col='zone_geom')
    
    # 2. Rename cột cho khớp DWH
    gdf = gdf.rename(columns={
        'zone_id': 'locationid',
        'zone_name': 'zonename',
        'borough': 'borough',
        'zone_geom': 'zonegeometry' # Đổi tên ở đây
    })
    
    # 🔥 QUAN TRỌNG: Cập nhật lại cột geometry active cho GeoPandas
    # Nếu không có dòng này, nó vẫn tìm 'zone_geom' và báo lỗi
    gdf = gdf.set_geometry('zonegeometry')
    
    # 3. Thêm cột SCD
    gdf['startdate'] = datetime(2024, 1, 1)
    gdf['enddate'] = None
    gdf['iscurrent'] = True
    gdf['servicezone'] = 'N/A'

    # 4. Đẩy vào DWH (PostGIS)
    try:
        gdf.to_postgis('dimlocation', engine_dwh, if_exists='append', index=False)
        print(f"✅ Đã nạp {len(gdf)} zones vào DimLocation.")
    except Exception as e:
        print(f"❌ Lỗi nạp DimLocation: {e}")

# ==============================================================================
# 4. HÀM INITIAL LOAD CHO CÁC DIMENSION KHÁC (SCD TYPE 2 INIT)
# ==============================================================================
def initial_load_scd():
    print("\n🚀 Đang thực hiện Initial Load cho các Dimension khác...")
    
    # --- A. DIM DRIVER (Từ Ops) ---
    print("   -> DimDriver...")
    df_driver = pd.read_sql("SELECT driver_id, legal_name, license_number, driver_status FROM drivers", engine_ops)
    
    # Mapping
    df_driver_dwh = pd.DataFrame()
    df_driver_dwh['driverid'] = df_driver['driver_id']
    df_driver_dwh['drivername'] = df_driver['legal_name']
    df_driver_dwh['licensenumber'] = df_driver['license_number']
    df_driver_dwh['driverstatus'] = df_driver['driver_status']
    
    # SCD Init
    df_driver_dwh['startdate'] = datetime(2024, 1, 1)
    df_driver_dwh['enddate'] = None
    df_driver_dwh['iscurrent'] = True
    
    df_driver_dwh.to_sql('dimdriver', engine_dwh, if_exists='append', index=False)

    # --- B. DIM VEHICLE (Từ Ops) ---
    print("   -> DimVehicle...")
    df_veh = pd.read_sql("SELECT vehicle_id, make_model, color, capacity FROM vehicles", engine_ops)
    
    df_veh_dwh = pd.DataFrame()
    df_veh_dwh['vehicleid'] = df_veh['vehicle_id']
    df_veh_dwh['vehiclemakemodel'] = df_veh['make_model']
    df_veh_dwh['vehiclecolor'] = df_veh['color']
    df_veh_dwh['vehiclecapacity'] = df_veh['capacity']
    df_veh_dwh['registrationyear'] = 2024 # Giả định
    
    # SCD Init
    df_veh_dwh['startdate'] = datetime(2024, 1, 1)
    df_veh_dwh['enddate'] = None
    df_veh_dwh['iscurrent'] = True
    
    df_veh_dwh.to_sql('dimvehicle', engine_dwh, if_exists='append', index=False)

    # --- C. DIM CUSTOMER (Từ CRM) ---
    print("   -> DimCustomer (Có thể lâu do dữ liệu lớn)...")
    # Đọc chunk nếu quá lớn, ở đây 600k dòng đọc 1 lần vẫn ổn với RAM 8GB (tầm 100MB RAM)
    df_cust = pd.read_sql("SELECT customer_id, display_name, phone_number, email, customer_segment, registration_date FROM customers", engine_crm)
    
    df_cust_dwh = pd.DataFrame()
    df_cust_dwh['customerid'] = df_cust['customer_id']
    df_cust_dwh['customername'] = df_cust['display_name']
    df_cust_dwh['phonenumber'] = df_cust['phone_number']
    df_cust_dwh['email'] = df_cust['email']
    df_cust_dwh['customersegment'] = df_cust['customer_segment']
    df_cust_dwh['registrationdate'] = df_cust['registration_date']
    
    # Xử lý ngày tháng
    df_cust_dwh['registrationdate'] = pd.to_datetime(df_cust_dwh['registrationdate'])
    df_cust_dwh['registrationyear'] = df_cust_dwh['registrationdate'].dt.year
    df_cust_dwh['registrationquarter'] = df_cust_dwh['registrationdate'].dt.to_period('Q').astype(str)
    
    # Tính CustomerAgeOnPlatform (Ví dụ tính đến hiện tại)
    current_year = datetime.now().year
    df_cust_dwh['customerageonplatform'] = current_year - df_cust_dwh['registrationyear']
    
    # SCD Init
    df_cust_dwh['startdate'] = datetime(2024, 1, 1)
    df_cust_dwh['enddate'] = None
    df_cust_dwh['iscurrent'] = True
    
    df_cust_dwh.to_sql('dimcustomer', engine_dwh, if_exists='append', index=False, chunksize=10000)

    # --- D. DIM PROMOTION (Từ CRM) ---
    print("   -> DimPromotion...")
    df_promo = pd.read_sql("SELECT promotion_id, promo_code, description, discount_value, discount_type, start_date, end_date FROM promotions", engine_crm)
    
    df_promo_dwh = pd.DataFrame()
    df_promo_dwh['promotionid'] = df_promo['promotion_id']
    df_promo_dwh['promotioncode'] = df_promo['promo_code']
    df_promo_dwh['promotionname'] = df_promo['promo_code'] # Tạm dùng code làm tên
    df_promo_dwh['description'] = df_promo['description']
    df_promo_dwh['discountvalue'] = df_promo['discount_value']
    df_promo_dwh['discounttype'] = df_promo['discount_type']
    
    # Tính Duration
    df_promo['start_date'] = pd.to_datetime(df_promo['start_date'])
    df_promo['end_date'] = pd.to_datetime(df_promo['end_date'])
    df_promo_dwh['durationindays'] = (df_promo['end_date'] - df_promo['start_date']).dt.days
    
    df_promo_dwh['startdate'] = df_promo['start_date']
    df_promo_dwh['enddate'] = df_promo['end_date']
    df_promo_dwh['promotionstatus'] = 'Active' # Giả định ban đầu
    
    df_promo_dwh.to_sql('dimpromotion', engine_dwh, if_exists='append', index=False)

    print("✅ Initial Load hoàn tất!")

# ==============================================================================
# 4.D. HÀM NẠP DIM PROMOTION (Cập nhật theo schema mới)
# ==============================================================================
def populate_dim_promotion():
    print("\n🎁 Đang nạp DimPromotion (Schema mới)...")
    
    # 1. Đọc dữ liệu từ CRM
    sql = """
        SELECT 
            promotion_id, promo_code, description, 
            discount_value, discount_type, 
            start_date, end_date, is_active 
        FROM promotions
    """
    df_promo = pd.read_sql(sql, engine_crm)
    
    if df_promo.empty:
        print("⚠️ Không có dữ liệu Promotion nào trong CRM.")
        return

    # 2. Mapping cột sang DWH
    df_dwh = pd.DataFrame()
    df_dwh['promotionid'] = df_promo['promotion_id']
    df_dwh['promotioncode'] = df_promo['promo_code']
    df_dwh['promotionname'] = df_promo['promo_code'] # Dùng code làm tên luôn
    df_dwh['description'] = df_promo['description']
    df_dwh['discounttype'] = df_promo['discount_type']
    df_dwh['discountvalue'] = df_promo['discount_value']
    
    # Chuyển đổi sang datetime chuẩn
    df_dwh['startdate'] = pd.to_datetime(df_promo['start_date'])
    df_dwh['enddate'] = pd.to_datetime(df_promo['end_date'])
    
    # 3. Tính toán các cột dẫn xuất (Derived Columns)
    
    # Campaign: Nguồn không có, gán mặc định
    df_dwh['campaign'] = 'General Campaign' 
    
    # DurationInDays: Số ngày hiệu lực
    df_dwh['durationindays'] = (df_dwh['enddate'] - df_dwh['startdate']).dt.days
    
    # PromotionStatus: Logic xác định trạng thái
    # Nếu is_active trong nguồn là 1 -> 'Active', ngược lại 'Inactive'
    # Hoặc so sánh ngày. Ở đây dùng cột is_active của nguồn cho chính xác.
    df_dwh['promotionstatus'] = df_promo['is_active'].apply(lambda x: 'Active' if x == 1 else 'Inactive')

    # 4. Đẩy vào DWH
    # Lưu ý: Không map cột iscurrent nữa vì schema không có
    try:
        df_dwh.to_sql('dimpromotion', engine_dwh, if_exists='append', index=False)
        print(f"✅ Đã nạp {len(df_dwh)} dòng vào DimPromotion.")
    except Exception as e:
        print(f"❌ Lỗi nạp Promotion: {e}")

# ==============================================================================
# 5. THỰC THI
# ==============================================================================
if __name__ == "__main__":
    # populate_dim_date()
    populate_dim_location()
    initial_load_scd()
    populate_dim_promotion()
    print("\n🎉 DWH CỦA BẠN ĐÃ CÓ DỮ LIỆU NỀN TẢNG ĐỂ CHẠY ETL FACT TABLE!")

🔗 Đã kết nối 3 Database thành công!

🌍 Đang nạp DimLocation từ Ops...
✅ Đã nạp 260 zones vào DimLocation.

🚀 Đang thực hiện Initial Load cho các Dimension khác...
   -> DimDriver...
   -> DimVehicle...
   -> DimCustomer (Có thể lâu do dữ liệu lớn)...
   -> DimPromotion...
✅ Initial Load hoàn tất!

🎁 Đang nạp DimPromotion (Schema mới)...
✅ Đã nạp 50 dòng vào DimPromotion.

🎉 DWH CỦA BẠN ĐÃ CÓ DỮ LIỆU NỀN TẢNG ĐỂ CHẠY ETL FACT TABLE!
